# Título

In [1]:
import sys
import os
sys.path.append('../..')

# Data Manipulation
import pandas as pd
import openpyxl

In [2]:
DATA_PATH = "../../data/"

## 1. Arquivos de Layout

### 1.1. Censo 2010

In [3]:
# Carrega o arquivo de labels como um arquivo Excel
layout_2010 = pd.ExcelFile('2010_Layout_microdados_Amostra.xlsx')

In [53]:
# Funções Auxiliares

def remove_unnamed_columns(df) -> pd.DataFrame:
    """
    Remove colunas com nome Unnamed. Estas colunas vem de colunas que foram mescladas no .xlsx original.

    Args:
        df (pd.DataFrame): DataFrame com colunas Unnamed

    Returns:
        pd.DataFrame: DataFrame sem colunas Unnamed
    
    """
    return df.loc[:, ~df.columns.str.contains('^Unnamed')]


def get_values(split: list):
    values = {}
    for val in range(1, len(split)):
        aux = split[val].split('-')
        if len(aux) == 1:
            aux.append(aux[0])
            aux[0] = ' '    
        # aux[1] = aux[1].replace(' ', '')
        values[aux[0]] = aux[1]
    # print(values)
    return values


def split_col_names_values(col: pd.Series): 
    df = pd.DataFrame(columns = ['VAR', 'NOME', 'CHAVE', 'VALOR'])
    for index, value in col.items():
        df_aux = pd.DataFrame(columns = df.columns)
        split = value.split('\n')
        # df_aux['NOME'] = pd.concat([df_aux['NOME'], pd.Series(split[0])])
        if len(split) > 1:
            # series_values = pd.concat([series_values, pd.Series(get_values(split))])
            # df_line = pd.DataFrame(columns = ['NOME', 'CHAVES', 'VALORES'])
            # # df_line['NOME'] = pd.Series(split[0])
            # df_line = pd.Series(get_values)
            # dct = pd.Series(get_values)
            lst = [{'CHAVE': d_key, 'VALOR': d_value, 'VAR': index} for d_key, d_value in get_values(split).items()]
            df_aux = pd.DataFrame(lst)
            df_aux['NOME'] = split[0]
            
        else:
            df_aux['NOME'] = pd.Series(split[0])
            
        df = pd.concat([df, df_aux])
    return df


# def prepare_df(df: pd.DataFrame):
#     df_main = pd.DataFrame(columns = df.columns)
#     df = remove_unnamed_columns(df) 
#     for index, row in df.iterrows():
#         df_row = pd.DataFrame(columns = df.columns)
#         # for col in df.columns.drop('NOME'):
#         #     # print(entry)
#         #     df_row[col] = df[col]
#         df_row = pd.concat([df_row, split_col_names_values(row['NOME'])])
#         # print(df_entry)
#         df_aux = pd.concat([df_aux, df_row])
#     return df, df_aux

def prepare_df(df: pd.DataFrame)-> tuple:
    df = remove_unnamed_columns(df)
    df_vars = split_col_names_values(df.set_index('VAR')['NOME'])
    return df, df_vars

In [55]:
a = prepare_df(df_domi)[0]
a

,VAR,NOME,POSIÇÃO INICIAL,POSIÇÃO FINAL,INT,DEC,TIPO
0,V0001,UNIDADE DA FEDERAÇÃO:\n11- Rondônia\n12- Acre\...,1,2,2,NaN,A
1,V0002,CÓDIGO DO MUNICÍPIO,3,7,5,NaN,A
2,V0011,ÁREA DE PONDERAÇÃO,8,20,13,NaN,A
3,V0300,CONTROLE,21,28,8,NaN,N
4,V0010,PESO AMOSTRAL,29,44,3,13.0,N
...,...,...,...,...,...,...,...
71,M0301,MARCA DE IMPUTAÇÃO NA V0301: \n1- Sim\n2- Não,168,168,1,NaN,C
72,M0401,MARCA DE IMPUTAÇÃO NA V0401: \n1- Sim\n2- Não,169,169,1,NaN,C
73,M0402,MARCA DE IMPUTAÇÃO NA V0402: \n1- Sim\n2- Não,170,170,1,NaN,C
74,M0701,MARCA DE IMPUTAÇÃO NA V0701: \n1- Sim\n2- Não,171,171,1,NaN,C


#### 1.1.1. Amostra de Domicílios

In [6]:
# Carrega as labels referentes aos microdados de domicílios
df_domi = pd.read_excel(layout_2010, sheet_name='DOMI')
# df_domi = remove_unnamed_columns(df_domi)
df_domi.head()

,VAR,NOME,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,POSIÇÃO INICIAL,POSIÇÃO FINAL,INT,DEC,TIPO
0,V0001,UNIDADE DA FEDERAÇÃO:\n11- Rondônia\n12- Acre\...,NaN,NaN,NaN,NaN,NaN,1,2,2,NaN,A
1,V0002,CÓDIGO DO MUNICÍPIO,NaN,NaN,NaN,NaN,NaN,3,7,5,NaN,A
2,V0011,ÁREA DE PONDERAÇÃO,NaN,NaN,NaN,NaN,NaN,8,20,13,NaN,A
3,V0300,CONTROLE,NaN,NaN,NaN,NaN,NaN,21,28,8,NaN,N
4,V0010,PESO AMOSTRAL,NaN,NaN,NaN,NaN,NaN,29,44,3,13.0,N


#### Outros

In [ ]:
df_domi.columns

Index(['VAR', 'NOME', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'POSIÇÃO INICIAL', 'POSIÇÃO FINAL', 'INT', 'DEC', 'TIPO'],
      dtype='object')

In [ ]:

# Dataframe de pessoas
df_pess = pd.read_excel(layout_2010, sheet_name='PESS')
# Dataframe de emigração
df_emig = pd.read_excel(layout_2010, sheet_name='EMIG')
# Dataframe de mortalidade
df_mort = pd.read_excel(layout_2010, sheet_name='MORT')

In [ ]:

data = {
    '1': 'a',
    '4': 'b',
    'c': 'v'
}

# Convert dictionary items to a list of dictionaries
list_of_dicts = [{'key': k, 'value': v} for k, v in data.items()]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(list_of_dicts)
df

,key,value
0,1,a
1,4,b
2,c,v
